In [3]:
from collections import defaultdict
import numpy as np
import sys
from enum import Enum
class ACTION(Enum):
    high='high'
    low='low' 
    
class Winner:
    def __init__(self):
        self.P_H = 0.55
        self.P_L = 0.45
        self.state = 0
        self.done = 0
        self.R = 0
        self.highcost = -50
        self.lowcost = -10
        self.greedy = 0.1
        self.discount=0.9
        self.init_Q()
        self.init_C()
    def reset_game(self):
        self.state=0
        self.done=0
    def ba_policy(self):
        smp = np.random.uniform(0,1)
        if smp <= 0.5:
            ac = ACTION.high
        else:
            ac = ACTION.low
        return ac
        
    #def ta_policy(self):
    def OffpolicyMCES(self,numepisode):
        self.init_Q()
        self.init_C()
        for i_episode in range(1, numepisode+1):
            if i_episode % 1000 == 0:
                print("\rEpisode {}/{}.".format(i_episode, numepisode), end="")
                sys.stdout.flush()
            
            self.reset_game()
            state=0
            episode = []
            while(1):
                action = self.ba_policy()
                reward,next_state,done = self.env_step(state,action)
                episode.append((state, action, reward))
                if done:
                    break
                state = next_state
            G = 0.0
            Weight = 1.0
            for t in reversed(range(len(episode))):
                state, action, reward = episode[t]
                G = self.discount * G + reward
                self.C[state,action] += Weight
                self.Q[state,action] += (Weight / self.C[state,action]) * (G - self.Q[state,action])
                if state > 1:
                    if action == ACTION.low:
                        Weight = Weight * (2.0)
                    else:
                        Weight=0
                        break
                else:
                    if action == ACTION.high:
                        Weight = Weight*(2.0)
                    else:
                        Weight=0
                        break
            
           
        return self.Q
        
        
    def env_step(self,state,action):
        done=0
        reward=0
        sample = np.random.uniform(0, 1)
        if action == ACTION.high:
            reward = reward + self.highcost
            suc_prob = self.P_H
        else:
            suc_prob = self.P_L
            reward = reward + self.lowcost
        if sample < suc_prob:
            state = state + 1
        else:
            state = state -1
        if state >= 3 :
            reward = reward + 1000
            done = 1
        if state <= (-3):
            done = 1
        return reward,state,done
    def init_Q(self):
        self.Q = defaultdict(lambda:np.zeros(1))
        for state in range(7):
            for act in ACTION:
                    self.Q[state-3,act]=0.0
    def init_C(self):
        
        self.C = defaultdict(lambda:np.zeros(1))
        for state in range(7):
            for act in ACTION:
                    self.C[state-3,act]=0.0
        

In [4]:
winner=Winner()

Q=winner.OffpolicyMCES(500000)
print('policy_value',Q)

Episode 500000/500000.policy_value defaultdict(<function Winner.init_Q.<locals>.<lambda> at 0x7f61c8fdb510>, {(-3, <ACTION.high: 'high'>): 0.0, (-3, <ACTION.low: 'low'>): 0.0, (-2, <ACTION.high: 'high'>): -42.91412287383462, (-2, <ACTION.low: 'low'>): -5.466951875974633, (-1, <ACTION.high: 'high'>): 2.39207895799618, (-1, <ACTION.low: 'low'>): 23.965096300374864, (0, <ACTION.high: 'high'>): 131.12490932164587, (0, <ACTION.low: 'low'>): 113.05822976943128, (1, <ACTION.high: 'high'>): 324.2667824821298, (1, <ACTION.low: 'low'>): 256.3376387369476, (2, <ACTION.high: 'high'>): 650.8311714200062, (2, <ACTION.low: 'low'>): 567.0397381037437, (3, <ACTION.high: 'high'>): 0.0, (3, <ACTION.low: 'low'>): 0.0})
